In [ ]:
%reset

# multi-gaussian fitting

Liz Strong 8/28/2019

In [ ]:
import sys
sys.path.append('../slvel')

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import set_matplotlib_formats
import pandas as pd
import seaborn as sns
from time import process_time
import fitting_functions as ff

### make pretty plots

In [ ]:
%matplotlib notebook

###  Load data

In [ ]:
intensities, times, ang_vels, orbit_radii, x_offsets, y_offsets, direction, noise_mean, noise_threshold, number_segments_per_burst = np.load(r"example_bursts.npy",allow_pickle=True) 
print("number of segments:", len(ang_vels))
print("number of different segments of different lengths made for each burst:", number_segments_per_burst)
print("number of bursts:", len(ang_vels)//number_segments_per_burst)
print("angular velocities:", np.unique(ang_vels))

# Fit bank of Gaussians to function

$g(t,\theta)=A e^{\frac{(t-\mu)^2}{2\sigma^2}}$, where $\theta=[A,\mu,\sigma]$

Model: $f_{\textrm{multi gaussian}}=\sum_{i=1}^{M}g_i(t,\theta_i)$

In [ ]:
t1_start = process_time() 

chars_to_pickle = {}
#del(characterizations_for_ml)

for selxn in range(2):
    print('fitting burst #', selxn)

    t, y, noise_avg, noise_thresh, ang_vel, dirctn = times[selxn], intensities[selxn], noise_mean[selxn], noise_threshold[selxn], ang_vels[selxn], direction[selxn]
    data = ff.fitting_function(selxn, t, y, noise_avg, noise_thresh, ang_vel, orbit_radii[selxn], x_offsets[selxn], y_offsets[selxn], dirctn, max_num_gaussians=8)

    try:
        chars_to_pickle[selxn] = data
    except:
        pass

    
c2p = {k: v for k, v in chars_to_pickle.items() if v is not None}
characterizations_for_ml = pd.DataFrame.from_dict(c2p,orient='index')
t1_stop = process_time() 
print("Elapsed time during the whole program in seconds:", 
                                         t1_stop-t1_start)  

In [ ]:
characterizations_for_ml.rename(index=str, columns={0: "ang_vel"},inplace=True)
characterizations_for_ml.rename(index=str, columns={1: "RMSE"},inplace=True)
characterizations_for_ml.rename(index=str, columns={2: "r2"},inplace=True)
characterizations_for_ml.rename(index=str, columns={3: "max_error"},inplace=True)
characterizations_for_ml.rename(index=str, columns={4: "max_error_normalized"},inplace=True)
characterizations_for_ml.rename(index=str, columns={5: "rect_effective_time"},inplace=True)
characterizations_for_ml.rename(index=str, columns={6: "duration"},inplace=True)
characterizations_for_ml.rename(index=str, columns={33: "orbit_R"},inplace=True)
characterizations_for_ml.rename(index=str, columns={34: "offsetx"},inplace=True)
characterizations_for_ml.rename(index=str, columns={35: "offsety"},inplace=True)
characterizations_for_ml.rename(index=str, columns={36: "D"},inplace=True)
characterizations_for_ml.rename(index=str, columns={37: "theta"},inplace=True)
characterizations_for_ml.rename(index=str, columns={38: "max_SNR"},inplace=True)
characterizations_for_ml.rename(index=str, columns={39: "avg_SNR"},inplace=True)
characterizations_for_ml.rename(index=str, columns={40: "clockwise"},inplace=True)
characterizations_for_ml.rename(index=str, columns={41: "counterclockwise"},inplace=True)
characterizations_for_ml.rename(index=str, columns={42: "num_initial_gaussians"},inplace=True)\

pd.set_option('display.max_columns', None)
characterizations_for_ml.tail()

### Pickle data for ML

In [ ]:
characterizations_for_ml.to_pickle('example_data.pkl')